In [1]:
import numpy as np
import pandas as pd
import yaml
import cv2
import os
import glob
import xml.etree.ElementTree as ET

In [2]:
root_folder = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations'
#Get all of the csv files that contain the anotation infromation for a single annotation clip 
annotation_file_list = glob.glob(root_folder + '/*.csv', recursive=True)
print(len(annotation_file_list), 'clips found.')

80 clips found.


In [3]:
annotation_file_list[:10]


['/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0018_annotation_322_170001850015_data.csv',
 '/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0019_annotation_323_1000250015_data.csv',
 '/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0019_annotation_324_95001100015_data.csv',
 '/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0020_annotation_325_1000250015_data.csv',
 '/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0020_annotation_326_120001350015_data.csv',
 '/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0021_annotation_415_165001800015_data.csv',
 '/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_

In [4]:
df = pd.read_csv(annotation_file_list[0])
df.head()

,index,annotation_id,class,date,frame_count,frame_number,frame_timestamp,name,oid,type,...,x0,x1,x2,x3,xform,y,y0,y1,y2,y3
0,0,322,buffalo_adult,NaN,17000,NaN,NaN,buffalo_adult_0,0,BVIEW_rectangle,...,2858,2937,2937,2858,False,1205.5,1159,1159,1252,1252
1,1,322,buffalo_adult,NaN,17015,NaN,NaN,buffalo_adult_0,0,BVIEW_rectangle,...,2858,2937,2937,2858,False,1205.5,1159,1159,1252,1252
2,2,322,buffalo_adult,NaN,17030,NaN,NaN,buffalo_adult_0,0,BVIEW_rectangle,...,2857,2936,2936,2857,False,1206.5,1160,1160,1253,1253
3,3,322,buffalo_adult,NaN,17045,NaN,NaN,buffalo_adult_0,0,BVIEW_rectangle,...,2856,2935,2935,2856,False,1208.5,1162,1162,1255,1255
4,4,322,buffalo_adult,NaN,17060,NaN,NaN,buffalo_adult_0,0,BVIEW_rectangle,...,2856,2935,2935,2856,False,1208.5,1162,1162,1255,1255


In [5]:
len(df['frame_count'].unique())

100

In [6]:
#Get all the video names assosiated with the annotations

video_names = []
for annotation_file in annotation_file_list:
    file_name = os.path.basename(annotation_file)
    split_file_name = file_name.split('_')
    video_name = split_file_name[1] + '_' + split_file_name[2] +'_' + split_file_name[3] +'_' + split_file_name[4] + '_' + split_file_name[5] +'.MOV'
    video_names.append(video_name)
    

In [7]:
annotation_df = pd.read_csv(annotation_file_list[0])
frames_list = annotation_df['frame_count']
frame_num = frames_list[10]
for row in annotation_df.index[annotation_df.loc[:, 'frame_count'] == frame_num].tolist()[:3]:
    #name
    name = annotation_df.loc[row, 'name']
    name = name.split('_')
    animal_name = name[0]
#             animal_age = name[1]
    print('animal_name:', animal_name)
    #pose


#             polygon_list = animal.findall('polygon')
#             points = polygon_list[frame].findall('pt')
#             top_left = [(points[0].find('x').text), (points[0].find('y').text)]
#             bottom_right = [(points[2].find('x').text), (points[2].find('y').text)]

    #bndbox

    print('x0:', str(annotation_df.loc[row, 'x0']))

    print('y0:', str(annotation_df.loc[row, 'y0']))

    print('x1:', str(annotation_df.loc[row, 'x1']))

    print('y1:', str(annotation_df.loc[row, 'y2']))

animal_name: buffalo
x0: 2857
y0: 1163
x1: 2936
y1: 1256
animal_name: buffalo
x0: 2615
y0: 1486
x1: 2708
y1: 1607
animal_name: buffalo
x0: 2501
y0: 1596
x1: 2571
y1: 1708


In [8]:
pd.options.display.max_columns = 100


output_folder = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/generated_xmls'  

video_root_folder =  '/home/golden/server-kenya_drones/raw_footage' 
 

annotations_folder_path = os.path.join(output_folder, 'annotations')

if not os.path.exists(annotations_folder_path):
    os.makedirs(annotations_folder_path)


for annotation_num, annotation_file in enumerate(annotation_file_list):

    annotation_df = pd.read_csv(annotation_file)
    video_name = video_names[annotation_num]
    try:
        video_file = glob.glob(video_root_folder + '/**/' + video_name, recursive=True)[0]
        print(video_file)
    except Exception as e:
        print(e)
        print('video_name', video_name)
        print('video_file', annotation_file)
        continue
    try:
        frames_list = annotation_df['frame_count'].unique()
        cap = cv2.VideoCapture(video_file)
        if cap is None or not cap.isOpened():
            raise Exception('Opencv unable to open the video file for %s' %video_file)

        video_name = video_name.split('.')[0]

        for frame_num, frame in enumerate(frames_list):
            cap.set(getattr(cv2, 'CAP_PROP_POS_FRAMES', 1), frame)
            _, image = cap.read()
            frame_name = video_name + '-' + str(frame) 
            frame_file = os.path.join(output_folder, frame_name + '.jpg')
            cv2.imwrite(frame_file, image)


            new_xml = ET.Element('annotation')
            folder = ET.SubElement(new_xml, 'folder')
            folder.text = 'annotated-frames'
            filename = ET.SubElement(new_xml, 'filename')
            filename.text = frame_name + '.jpg'
            image_size = ET.SubElement(new_xml, 'size')
            image_width = ET.SubElement(image_size, 'width')
            image_width.text = str(image.shape[1])
            image_height = ET.SubElement(image_size, 'height')
            image_height.text = str(image.shape[0])
            image_depth = ET.SubElement(image_size, 'depth')
            image_depth.text = str(image.shape[2])

            segmented = ET.SubElement(new_xml, 'segmented')
            segmented.text = '0'
            for row in annotation_df.index[annotation_df.loc[:, 'frame_count'] == frame].tolist():
                animal_object = ET.SubElement(new_xml, 'object')
                #name
                name = annotation_df.loc[row, 'name']
                name = name.split('_')
                animal_name = name[0]
    #             animal_age = name[1]
                name = ET.SubElement(animal_object, 'name')
                name.text = animal_name
                #pose
                pose = ET.SubElement(animal_object, 'pose')
                pose.text = 'Unspecified'
                #truncated
                truncated = ET.SubElement(animal_object, 'truncated')
                truncated.text = '0'
                #difficult
                difficult = ET.SubElement(animal_object, 'difficult')
                difficult.text = '0'

    #             polygon_list = animal.findall('polygon')
    #             points = polygon_list[frame].findall('pt')
    #             top_left = [(points[0].find('x').text), (points[0].find('y').text)]
    #             bottom_right = [(points[2].find('x').text), (points[2].find('y').text)]

                #bndbox
                bndbox = ET.SubElement(animal_object, 'bndbox')
                xmin = ET.SubElement(bndbox, 'xmin')
                xmin.text = str(annotation_df.loc[row, 'x0'])
                ymin = ET.SubElement(bndbox, 'ymin')
                ymin.text = str(annotation_df.loc[row, 'y0'])
                xmax = ET.SubElement(bndbox, 'xmax')
                xmax.text = str(annotation_df.loc[row, 'x1'])
                ymax = ET.SubElement(bndbox, 'ymax')
                ymax.text = str(annotation_df.loc[row, 'y2'])

                #age
    #             age = ET.SubElement(animal_object, 'age')
    #             age.text = animal_age 
            outfile = os.path.join(output_folder, 'annotations/' + frame_name + str('.xml'))
            if frame_num == 0:
                print('outfile', outfile)
            tree = ET.ElementTree(new_xml)
            tree.write(outfile)
    except:
        print('annotation file ', annotation_file, 'failed...')
        continue
        

cap.release()
        

        
        
        





list index out of range
video_name NOV02_2017_DJI_0018_annotation.MOV
video_file /media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0018_annotation_322_170001850015_data.csv
list index out of range
video_name NOV02_2017_DJI_0019_annotation.MOV
video_file /media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0019_annotation_323_1000250015_data.csv
list index out of range
video_name NOV02_2017_DJI_0019_annotation.MOV
video_file /media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0019_annotation_324_95001100015_data.csv
list index out of range
video_name NOV02_2017_DJI_0020_annotation.MOV
video_file /media/golden/72FFC6EE48B5CF39/kenya-tracking/loopy-annotations/exported_annotations/video_NOV02_2017_DJI_0020_annotation_325_1000250015_data.csv
list index out of range
video_name NOV02_2017_DJI_0020_annotation.MOV
video_file /media/g

In [9]:
file = '/home/golden/server-kenya_drones/loopy_annotations/v1/videos/annotated/object-detection/by_key/video=830/clip=12000_13500_15/annotations.feather' 
df = pd.read_feather(file)
df.head()

,auto_obj_id,obj_id,frame_num,manual,min_col,min_row,max_col,max_row,name,origin
0,0,ID_0,0,True,1907,564,2007,646,buffalo_adult,b'human'
1,0,ID_0,1,False,1899,559,1999,641,buffalo_adult,b'human'
2,0,ID_0,2,False,1889,554,1989,636,buffalo_adult,b'human'
3,0,ID_0,3,False,1878,549,1978,631,buffalo_adult,b'human'
4,0,ID_0,4,False,1870,544,1970,626,buffalo_adult,b'human'


In [ ]:
yaml_file = '/Volumes/kenya_drones/loopy_annotations/v1/videos/annotated/object-detection/by_key/video=830/clip=12000_13500_15/info.yaml'
with open(yaml_file, 'r') as f:
    yaml_df = pd.io.json.json_normalize(yaml.load(f))
pd.options.display.max_columns = 100
print(pd.options.display.max_columns)

yaml_df

In [ ]:
annotation_df.index[annotation_df.loc[:, 'frame_num'] == frame_num].tolist()


In [ ]:
video_file = '' 

# video_stream = cv2.VideoCapture(video_file)
# total_frames = video_stream.get(cv2.CAP_PROP_FRAME_COUNT)

annotated_frames = df.loc[:, 'frame_num'] .unique()

for frame in range(total_frames):
    if frame == annotated_frame
    video_stream.read()


print(annotated_frames[0])